In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Define the number of classes (occasion categories)
num_classes = 4

# Data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,  # Split the dataset into training and validation
    rotation_range=20,  # Example data augmentation options
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Load the pre-trained VGG16 model without top layers
base_model = VGG16(weights='imagenet', include_top=False)

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the classification model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data directory (organized by occasion)
data_dir = '/content/drive/MyDrive/Colab Notebooks/FYP/Recommendation System/polyvore'

# Flow training images in batches
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # Adjust image size as needed
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Flow validation images in batches
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Train the model
model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Save the model
model.save('occasion_classification_model.h5')


In [ ]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

# Load the trained model
model = tf.keras.models.load_model('occasion_classification_model.h5')

# Create a dictionary to map occasion labels to folder names
occasion_folders = {
    'Casual': 'casual_images',
    'Formal': 'formal_images',
    'Party': 'party_images',
    'Business': 'business_images'
}

# Path to the directory containing clothing images
images_directory = 'path_to_directory_with_clothing_images'  # Replace with your images directory

# Iterate through each image in the directory
for filename in os.listdir(images_directory):
    img_path = os.path.join(images_directory, filename)

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))  # Adjust the target size to match your model
    x = image.img_to_array(img)
    x = preprocess_input(x)
    x = tf.expand_dims(x, axis=0)  # Add a batch dimension

    # Perform classification
    predictions = model.predict(x)
    predicted_class = tf.argmax(predictions, axis=1).numpy()[0]

    # Map the class index to occasion labels
    occasion_labels = {0: 'Casual', 1: 'Formal', 2: 'Party', 3: 'Business'}
    predicted_occasion = occasion_labels[predicted_class]

    # Get the folder name for the occasion
    occasion_folder = occasion_folders.get(predicted_occasion, 'other_images')

    # Create the occasion folder if it doesn't exist
    occasion_folder_path = os.path.join(images_directory, occasion_folder)
    os.makedirs(occasion_folder_path, exist_ok=True)

    # Copy or move the image to the occasion folder
    shutil.copy(img_path, os.path.join(occasion_folder_path, filename))

print("Image classification and organization complete.")
